# Tugas Crawling PTA dan Berita


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time

In [2]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    # Cari tombol >> (last page)
    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        # Pecah URL -> ambil angka terakhir
        max_page = int(href.split("/")[-1])
        return max_page

    # fallback kalau pagination tidak ada
    return 1
print(get_max_page(10))

172


/home/codespace/.local/lib/python3.12/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


In [3]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

In [ ]:
def pta_all():
    start_time = time.time()

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak_id": [],
        "abstrak_en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": []
    }

    total_prodi = 1
    total_pages = 0
    max_pages_dict = {}

    # hitung total halaman (untuk tiap prodi)
    for i in range(1, total_prodi + 1):
        max_page = get_max_page(i)
        max_pages_dict[i] = max_page
        total_pages += max_page

    for i in range(1, total_prodi + 1):
        max_page = max_pages_dict[i]
        for j in range(1, max_page + 1):
            url = f"{BASE_URL}/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")

            for jurnal in jurnals:
                link_keluar = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA (angka terakhir di URL)
                id_match = re.search(r"/detail/(\d+)", link_keluar)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link_keluar)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                judul = isi.select_one('a.title').text.strip()
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                paragraf = isi.select('p[align="justify"]')
                abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["abstrak_id"].append(abstrak_id)
                data["abstrak_en"].append(abstrak_en)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["prodi"].append(prodi)

            # update progress bar per prodi
            print_progress(i, prodi, j, max_page)

        sys.stdout.write("\n")  # pindah baris setelah 1 prodi selesai

    # simpan ke CSV
    df = pd.DataFrame(data)
    df.to_csv("pta_all.csv", index=False, encoding="utf-8-sig")

    # hitung durasi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

pta_all()

[1] Ilmu Hukum - Page 284/284 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 1417
⏱️ Waktu eksekusi: 1 jam 24 menit 15 detik


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
2,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
1412,150111100130,DEDY DORES,PENGKUALIFIKASIAN CHEATER SEBAGAI TINDAK PIDAN...,Abstrak\n Perbuatan cheater dalam melakukan ch...,Abstract\n The way of cheater did a cheat in o...,"Aris Hardinanto, S.H., M.H.",,Ilmu Hukum
1413,150111100258,Eko Supriadi,KUALIFIKASI TINDAK PIDANA ATAS PERBUATAN PELAK...,Peminjaman dana sistem online dilakukan oleh m...,The loan funds by online system are carried ou...,"Dr. Erma Rusdiana, S.H.,M.H",,Ilmu Hukum
1414,160111100136,Muslimatul Maghfirah,KEDUDUKAN HUKUM PEKERJA OUTSOURCING DI DINAS P...,Abstrak\nTenaga kerja merupakan setiap orang y...,Abstract\nLabors are those who can work to pro...,"Mishbahul Munir, S.H., M.Hum",,Ilmu Hukum
1415,160111100024,MOH WASIL SYAHRONI,STAGNANSI HUBUNGAN KELEMBAGAAN DAN KEWENANGAN ...,Skripsi ini bertujuan untuk menganalisis penti...,This thesis aims to analyze the stagnation of ...,"Dr. DENI SETYA BAGUS YUHERAWAN, S.H., M.S",,Ilmu Hukum


# Fungsi Crawling Data PTA (Dengan Batas Page)

In [4]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

def pta():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "id": [],
        "penulis": [],
        "judul": [],
        "abstrak id": [],
        "abstrak en": [],
        "pembimbing_pertama": [],
        "pembimbing_kedua": [],
        "prodi": [],
    }

    for i in range(1, 42):  # jumlah prodi
        total_pages = 3  # jumlah page
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop page
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                # ambil ID dari link PTA
                id_match = re.search(r"/detail/(\d+)", link)
                pta_id = id_match.group(1) if id_match else None

                response = requests.get(link)
                soup1 = BeautifulSoup(response.content, "html.parser")
                isi = soup1.select_one('div#content_journal')

                # Judul
                judul = isi.select_one('a.title').text

                # Penulis
                penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]

                # Pembimbing Pertama
                pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]

                # Pembimbing Kedua
                pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

                # Abstrak
                paragraf = isi.select('p[align="justify"]')
                abstrak = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
                abstract = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

                # simpan data
                data["id"].append(pta_id)
                data["penulis"].append(penulis)
                data["judul"].append(judul)
                data["pembimbing_pertama"].append(pembimbing_pertama)
                data["pembimbing_kedua"].append(pembimbing_kedua)
                data["abstrak id"].append(abstrak)
                data["abstrak en"].append(abstract)
                data["prodi"].append(prodi)

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

pta()

[1] Ilmu Hukum - Page 3/3 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 3/3 [████████████████████] 100.00%

[3] Agribisnis - Page 3/3 [████████████████████] 100.00%

[4] Agroteknologi - Page 3/3 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 3/3 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 3/3 [████████████████████] 100.00%

[7] Manajemen - Page 3/3 [████████████████████] 100.00%

[8] Akuntansi - Page 3/3 [████████████████████] 100.00%

[9] Teknik Industri - Page 3/3 [████████████████████] 100.00%

[10] Teknik Informatika - Page 3/3 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 3/3 [████████████████████] 100.00%

[12] Sosiologi - Page 3/3 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 3/3 [████████████████████] 100.00%

[14] Psikologi - Page 3/3 [████████████████████] 100.00%

[15] Sastra Inggris - Page 3/3 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 3/3 [████████████████████] 100.00%

,id,penulis,judul,abstrak id,abstrak en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
2,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
476,160281100013,"Lisa Sri rahmatullah, S. Sos. I",Dampak Sosial Ekonomi Pariwisata Religi Makam ...,Penelitian ini bertujuan untuk mengetahui baga...,The purpose of this study is to analyze the so...,"Dr. Diah Wahyuningsih, S.E., M.Si.","Dr. Eni Sri Rahayuningsih, S.E., M.E.",Magister Ilmu Ekonomi
477,160281100002,Indah Ainun Nikmah,Peranan Zakat Produktif Dalam Meningkatkan Eko...,Peranan Zakat Produktif dalam Meningkatkan Eko...,The Role of Productive Zakat in Improving Must...,"Dr. Kurniyati Indahsari, M.Si","Dr. Abdur Rahman, S.Ag. MEI",Magister Ilmu Ekonomi
478,170361100010,ahmad syaiful umam,KARAKTERISASI DAN KOLEKSI PLASMA NUTFAH UNTUK ...,Madura merupakan salah satu wilayah pemasok ko...,Madura is one of the regions supplying horticu...,"Dr. Ir. Gita Pawana, M.Si","Dr. Ir. Hj. SIti Fatimah, M.Si",Magister Pengelolaan Sumber Daya Alam
479,170361100001,Siti Holifah,PENGOLAHAN LIMBAH AIR REBUSAN IKAN TERI MENJAD...,Ikan Teri perlu penanganan serius pasca panen ...,Anchovy needs serious handling after harvest b...,"Dr.Apri Arisandi,S.Pi.,M.Si.","Dr.Ir.H.Asfan,MP.",Magister Pengelolaan Sumber Daya Alam


# Page & Link Keluar PTA

In [5]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

def pta_links():
    start_time = time.time()  # mulai hitung waktu

    data = {
        "no": [],
        "page": [],
        "link_keluar": []
    }

    no = 1  # nomor urut

    for i in range(1, 42):  # jumlah prodi
        total_pages = 3  # jumlah page
        prodi_name = None

        for j in range(1, total_pages + 1):  # loop page
            url = f"https://pta.trunojoyo.ac.id/c_search/byprod/{i}/{j}"
            r = requests.get(url)
            soup = BeautifulSoup(r.content, "html.parser")
            jurnals = soup.select('li[data-cat="#luxury"]')

            isii = soup.select_one('div#begin')
            if not isii:
                continue
            prodi_full = isii.select_one('h2').text.strip()
            prodi = prodi_full.replace("Journal Jurusan ", "")
            if not prodi_name:
                prodi_name = prodi

            for jurnal in jurnals:
                link = jurnal.select_one('a.gray.button')['href']

                data["no"].append(no)
                data["page"].append(url)          # link page
                data["link_keluar"].append(link)  # link detail
                no += 1

            # update progress bar
            print_progress(i, prodi_name, j, total_pages)

    df = pd.DataFrame(data)
    df.to_csv("pta_links.csv", index=False)

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

pta_links()

[1] Ilmu Hukum - Page 3/3 [████████████████████] 100.00%

[2] Teknologi Industri Pertanian - Page 3/3 [████████████████████] 100.00%

[3] Agribisnis - Page 3/3 [████████████████████] 100.00%

[4] Agroteknologi - Page 3/3 [████████████████████] 100.00%

[5] Ilmu Kelautan - Page 3/3 [████████████████████] 100.00%

[6] Ekonomi Pembangunan - Page 3/3 [████████████████████] 100.00%

[7] Manajemen - Page 3/3 [████████████████████] 100.00%

[8] Akuntansi - Page 3/3 [████████████████████] 100.00%

[9] Teknik Industri - Page 3/3 [████████████████████] 100.00%

[10] Teknik Informatika - Page 3/3 [████████████████████] 100.00%

[11] Manajemen Informatika - Page 3/3 [████████████████████] 100.00%

[12] Sosiologi - Page 3/3 [████████████████████] 100.00%

[13] Ilmu Komunikasi - Page 3/3 [████████████████████] 100.00%

[14] Psikologi - Page 3/3 [████████████████████] 100.00%

[15] Sastra Inggris - Page 3/3 [████████████████████] 100.00%

[16] Ekonomi Syariah - Page 3/3 [████████████████████] 100.00%

,no,page,link_keluar
0,1,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
1,2,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/080...
2,3,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
3,4,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/090...
4,5,https://pta.trunojoyo.ac.id/c_search/byprod/1/1,https://pta.trunojoyo.ac.id/welcome/detail/070...
...,...,...,...
476,477,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
477,478,https://pta.trunojoyo.ac.id/c_search/byprod/36/2,https://pta.trunojoyo.ac.id/welcome/detail/160...
478,479,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...
479,480,https://pta.trunojoyo.ac.id/c_search/byprod/37/1,https://pta.trunojoyo.ac.id/welcome/detail/170...


## 2. Crawling Berita

In [17]:
# Fungsi progress bar manual
def print_progress(kategori, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r{kategori} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n\n')

In [18]:
# Fungsi untuk ambil konten artikel
def get_article_content(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")

    paragraphs = []
    content_divs = soup.find_all("div", id="content-wrapper")
    for div in content_divs:
        for p in div.find_all("p"):
            text = p.get_text(strip=True)
            if text and not text.lower().startswith("baca juga"):
                paragraphs.append(text)
    return " ".join(paragraphs)

In [19]:
def berita(categories, pages_per_category=1):
    start_time = time.time()  # mulai hitung waktu

    BASE_URL = "https://www.tempo.co/indeks?page={}&category=rubrik&rubric_slug={}"

    data = {
        "id_berita": [],
        "judul_berita": [],
        "isi_berita": [],
        "kategori_berita": []
    }

    for cat_id, cat in enumerate(categories, start=1):
        for page in range(1, pages_per_category+1):
            url = BASE_URL.format(page, cat)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            articles = soup.select("figure figcaption a")
            for a in articles:
                link = "https://www.tempo.co" + a["href"]
                title = a.get_text(strip=True)

                id_match = re.search(r"-(\d+)$", link)
                berita_id = id_match.group(1) if id_match else None

                try:
                    content = get_article_content(link)
                except:
                    content = ""

                data["id_berita"].append(berita_id)
                data["judul_berita"].append(title)
                data["isi_berita"].append(content)
                data["kategori_berita"].append(cat)

            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("tempo_berita.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

categories = ["politik", "hukum", "ekonomi", "lingkungan", "hiburan", "internasional", "otomotif", "olahraga", "sepakbola"]


berita(categories, pages_per_category=5)

politik - Page 5/5 [████████████████████] 100.00%

hukum - Page 5/5 [████████████████████] 100.00%

ekonomi - Page 5/5 [████████████████████] 100.00%

lingkungan - Page 5/5 [████████████████████] 100.00%

hiburan - Page 5/5 [████████████████████] 100.00%

internasional - Page 5/5 [████████████████████] 100.00%

otomotif - Page 5/5 [████████████████████] 100.00%

olahraga - Page 5/5 [████████████████████] 100.00%

sepakbola - Page 5/5 [████████████████████] 100.00%


✅ Seluruh data berhasil dikumpulkan!
📊 Total entri: 220
⏱️ Waktu eksekusi: 0 jam 2 menit 18 detik


,id_berita,judul_berita,isi_berita,kategori_berita
0,2070560,Habis Demo Berujung Rusuh Terbitlah Siskamling,ATAP berukuran 5 x 3 meter menaungi Jalan Kema...,politik
1,2070554,Siskamling: Warisan Orde Baru yang Dihidupkan ...,MENTERI Dalam NegeriTito Karnavianmeminta peme...,politik
2,2070547,6.118 Aparat Gabungan Disiagakan Jaga Demo di ...,ASOSIASI pengemudiojekonlineGarda Indonesia be...,politik
3,2070538,Prabowo Dikabarkan akan Melakukan Reshuffle Ka...,PRESIDENPrabowo Subiantodikabarkan akan melaku...,politik
4,2070529,Fakta Terbaru Soal Ompreng MBG Diduga Mengandu...,ISU dugaanompreng atau wadah makan bergizi gra...,politik
...,...,...,...,...
215,2070442,Kejagung Periksa Anak Jusuf Hamka soal Dugaan ...,"Kejaksaan Agung memeriksa Fitria Yusuf, anak p...",ekonomi
216,2070429,"Martubung, Kawasan Padat Penduduk di Medan yan...","TAMAN kolam retensi, tepat di samping Perumnas...",ekonomi
217,2070424,KAI Siapkan Promo Tarif Flat Rp 80 Ribu saat H...,PT KERETA Api Indonesia (Persero) atauKAIsecar...,ekonomi
218,2070420,KKP Bilang Penguggah Video Tanggul Beton di Ci...,,ekonomi


## Page & Link Keluar Berita

In [23]:
def berita_links(categories, pages_per_category=1):
    start_time = time.time()  # mulai hitung waktu

    BASE_URL = "https://www.tempo.co/indeks?page={}&category=rubrik&rubric_slug={}"

    data = {
        "id_berita": [],
        "page": [],
        "link_keluar": []
    }

    for cat in categories:
        for page in range(1, pages_per_category+1):
            url = BASE_URL.format(page, cat)
            r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(r.text, "html.parser")

            articles = soup.select("figure figcaption a")
            for a in articles:
                link = "https://www.tempo.co" + a["href"]
                id_match = re.search(r"-(\d+)$", link)
                berita_id = id_match.group(1) if id_match else None

                data["id_berita"].append(berita_id)
                data["page"].append(url)         # link page
                data["link_keluar"].append(link) # link detail

            # update progress bar
            print_progress(cat, page, pages_per_category)

    df = pd.DataFrame(data)
    df.to_csv("tempo_links.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # summary
    print("\n✅ Seluruh link berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

categories = ["politik", "hukum", "ekonomi", "lingkungan", "hiburan", "internasional", "otomotif", "olahraga", "sepakbola"]

berita_links(categories, pages_per_category=5)

politik - Page 1/5 [████----------------] 20.00%

politik - Page 5/5 [████████████████████] 100.00%

hukum - Page 5/5 [████████████████████] 100.00%

ekonomi - Page 5/5 [████████████████████] 100.00%

lingkungan - Page 5/5 [████████████████████] 100.00%

hiburan - Page 5/5 [████████████████████] 100.00%

internasional - Page 5/5 [████████████████████] 100.00%

otomotif - Page 5/5 [████████████████████] 100.00%

olahraga - Page 5/5 [████████████████████] 100.00%

sepakbola - Page 5/5 [████████████████████] 100.00%


✅ Seluruh link berhasil dikumpulkan!
📊 Total entri: 900
⏱️ Waktu eksekusi: 0 jam 0 menit 26 detik


,id_berita,page,link_keluar
0,2070560,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/aktivitas-siskaml...
1,2070554,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/tito-karnavian-hi...
2,2070547,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/6-118-aparat-gabu...
3,2070538,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/prabowo-dikabarka...
4,2070529,https://www.tempo.co/indeks?page=1&category=ru...,https://www.tempo.co/politik/fakta-terbaru-soa...
...,...,...,...
895,2068129,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/susunan-pemain-...
896,2068091,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/pelatih-lebanon...
897,2068046,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/erick-thohir-se...
898,2067975,https://www.tempo.co/indeks?page=5&category=ru...,https://www.tempo.co/sepakbola/kenapa-nuno-esp...
